In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext lab_black

In [3]:
from src.load_data import load_data

epoch_key = "bon", 3, 6

data = load_data(epoch_key)

Failed to load file: /data2/edeno/replay_trajectory_paper/src/../Raw-Data/Bond/bonDIO03.mat
No DIO file found, using distance from well to segment trials
Failed to load file: /data2/edeno/replay_trajectory_paper/src/../Raw-Data/Bond/bonDIO03.mat
No DIO file found, inferring correct inbound/outbound from task rules
Failed to load file: /data2/edeno/replay_trajectory_paper/src/../Raw-Data/Bond/bonDIO03.mat
No DIO file found, using distance from well to segment trials
Failed to load file: /data2/edeno/replay_trajectory_paper/src/../Raw-Data/Bond/bonDIO03.mat
No DIO file found, inferring correct inbound/outbound from task rules


In [136]:
import xarray as xr
from src.parameters import PROCESSED_DATA_DIR
import os

animal, day, epoch = epoch_key
data_type, dim = "clusterless", "1D"

results = xr.open_dataset(
    os.path.join(PROCESSED_DATA_DIR, f"{animal}_{day:02}_{epoch:02}.nc"),
    group=f"/{data_type}/{dim}/classifier/ripples/",
)

results

<xarray.Dataset>
Dimensions:            (position: 99, ripple_number: 193, state: 3, time: 184)
Coordinates:
  * position           (position) float64 1.46 4.38 7.3 ... 308.3 311.3 314.3
  * state              (state) object 'Continuous' 'Fragmented' 'Hover'
  * time               (time) timedelta64[ns] 00:00:00 ... 00:00:00.366000
  * ripple_number      (ripple_number) int64 1 2 3 4 5 6 ... 189 190 191 192 193
Data variables:
    acausal_posterior  (ripple_number, time, state, position) float64 ...

In [137]:
from loren_frank_data_processing import reshape_to_segments
from src.analysis import (
    get_probability,
    get_is_classified,
    get_n_unique_spiking,
    get_n_total_spikes,
)
import pandas as pd
import numpy as np

probability_threshold = 0.8
sampling_frequency = 500
ripple_times = data["ripple_times"].loc[:, ["start_time", "end_time"]]
spikes = (
    (((~np.isnan(data["multiunit"])).sum("features") > 0) * 1.0)
    .to_dataframe(name="spikes")
    .unstack()
)
spikes.columns = data["tetrode_info"].tetrode_id
ripple_spikes = reshape_to_segments(spikes, ripple_times)
position_info = data["position_info"]


probability = get_probability(results)
is_classified = get_is_classified(probability, probability_threshold)

duration = is_classified.sum("time") / sampling_frequency
duration = duration.to_dataframe().unstack(level=1)
duration.columns = list(duration.columns.get_level_values("state"))
duration = duration.rename(columns=lambda column_name: column_name + "_duration")
is_category = (duration > 0.0).rename(columns=lambda c: c.split("_")[0])
duration = pd.concat((duration, is_category), axis=1)
duration["is_classified"] = np.any(duration > 0.0, axis=1)
duration["n_unique_spiking"] = get_n_unique_spiking(ripple_spikes)
duration["n_total_spikes"] = get_n_total_spikes(ripple_spikes)
duration["population_rate"] = (
    ripple_spikes.groupby("ripple_number").mean().mean(axis=1) * sampling_frequency
)

ripple_position_info = reshape_to_segments(position_info, ripple_times)
duration["actual_x_position"] = ripple_position_info.groupby(
    "ripple_number"
).x_position.mean()
duration["actual_y_position"] = ripple_position_info.groupby(
    "ripple_number"
).y_position.mean()
duration["actual_linear_distance"] = ripple_position_info.groupby(
    "ripple_number"
).linear_distance.mean()
duration["actual_linear_position"] = ripple_position_info.groupby(
    "ripple_number"
).linear_position.mean()
duration["actual_speed"] = ripple_position_info.groupby("ripple_number").speed.mean()
duration["actual_velocity_center_well"] = ripple_position_info.groupby(
    "ripple_number"
).linear_velocity.mean()

duration

,Continuous_duration,Fragmented_duration,Fragmented-Continuous-Mix_duration,Hover_duration,Hover-Continuous-Mix_duration,Continuous,Fragmented,Fragmented-Continuous-Mix,Hover,Hover-Continuous-Mix,is_classified,n_unique_spiking,n_total_spikes,population_rate,actual_x_position,actual_y_position,actual_linear_distance,actual_linear_position,actual_speed,actual_velocity_center_well
ripple_number,,,,,,,,,,,,,,,,,,,,
1,0.016,0.002,0.014,0.046,0.080,True,True,True,True,True,True,21,509.0,106.396321,137.087740,116.647865,2.313181,2.313181,1.429294,NaN
2,0.000,0.000,0.000,0.000,0.068,False,False,False,False,True,True,18,158.0,79.878665,137.700000,119.176129,1.700922,1.700922,1.078215,NaN
3,0.000,0.038,0.006,0.000,0.012,False,True,True,False,True,True,20,242.0,125.258799,140.337500,117.900000,0.000000,0.000000,0.787650,2.048812
4,0.022,0.000,0.000,0.000,0.036,True,False,False,False,True,True,20,170.0,102.657005,140.400000,117.149306,0.000000,0.000000,1.196156,5.947335
5,0.000,0.044,0.008,0.000,0.098,False,True,True,False,True,True,21,617.0,145.793951,137.645380,153.208696,178.626949,313.949678,1.043303,16.162358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.062,0.014,0.016,0.000,0.052,True,True,True,False,True,True,20,422.0,97.594820,142.752926,115.650000,0.000000,0.000000,0.454844,2.841736
190,0.000,0.000,0.000,0.000,0.112,False,False,False,False,True,True,17,149.0,57.841615,142.911161,114.300000,0.000000,0.000000,0.755726,6.367183
191,0.026,0.000,0.000,0.000,0.126,True,False,False,False,True,True,20,361.0,92.327366,91.800000,117.000000,47.600922,47.600922,0.073890,0.058544


In [23]:
ripple_number = 1

In [107]:
from src.analysis import (
    maximum_a_posteriori_estimate,
    calculate_replay_distance,
    gaussian_smooth,
    highest_posterior_density,
)
from src.parameters import ANIMALS
from loren_frank_data_processing.position import (
    EDGE_ORDER,
    EDGE_SPACING,
    make_track_graph,
)

track_graph, _ = make_track_graph(epoch_key, ANIMALS)

posterior = (
    results.sel(ripple_number=ripple_number)
    .acausal_posterior.dropna("time", how="all")
    .assign_coords(time=lambda ds: ds.time / np.timedelta64(1, "s"))
)
is_classified = (
    is_classified.sel(ripple_number=ripple_number)
    .dropna("time", how="all")
    .assign_coords(time=lambda ds: ds.time / np.timedelta64(1, "s"))
)
probability = (
    probability.sel(ripple_number=ripple_number)
    .dropna("time", how="all")
    .assign_coords(time=lambda ds: ds.time / np.timedelta64(1, "s"))
)
ripple_spikes = ripple_spikes.loc[ripple_number]
map_estimate = maximum_a_posteriori_estimate(posterior.sum("state"))

actual_positions = ripple_position_info.loc[
    ripple_number, ["x_position", "y_position"]
].values
actual_track_segment_ids = (
    ripple_position_info.loc[ripple_number, "track_segment_id"]
    .values.squeeze()
    .astype(int)
)

(
    replay_distance_from_actual_position,
    replay_distance_from_center_well,
) = calculate_replay_distance(
    track_graph, map_estimate, actual_positions, actual_track_segment_ids, position_info
)

try:
    replay_total_displacement = np.abs(
        replay_distance_from_center_well[-1] - replay_distance_from_center_well[0]
    )
except IndexError:
    replay_total_displacement = np.nan

time = np.asarray(posterior.time)
map_estimate = map_estimate.squeeze()
replay_speed = np.abs(np.gradient(replay_distance_from_center_well, time))
SMOOTH_SIGMA = 0.0025
replay_speed = gaussian_smooth(replay_speed, SMOOTH_SIGMA, sampling_frequency)
replay_velocity_actual_position = np.gradient(
    replay_distance_from_actual_position, time
)
replay_velocity_center_well = np.gradient(replay_distance_from_center_well, time)

hpd_threshold = highest_posterior_density(posterior.sum("state"), coverage=0.95)
isin_hpd = posterior.sum("state") >= hpd_threshold[:, np.newaxis]
spatial_coverage = (isin_hpd * np.diff(posterior.position)[0]).sum("position").values
n_position_bins = (posterior.sum("state", skipna=True) > 0).sum("position").values[0]
spatial_coverage_percentage = (isin_hpd.sum("position") / n_position_bins).values
distance_change = np.abs(np.diff(replay_distance_from_center_well))
distance_change = np.insert(distance_change, 0, 0)

In [140]:
import scipy
import itertools


def get_replay_linear_position_by_state(
    ripple_number, is_classified, results, state="Hover"
):
    posterior = (
        results.sel(ripple_number=ripple_number)
        .acausal_posterior.dropna("time", how="all")
        .assign_coords(time=lambda ds: ds.time / np.timedelta64(1, "s"))
    )
    is_classified = (
        is_classified.sel(ripple_number=ripple_number)
        .dropna("time", how="all")
        .assign_coords(time=lambda ds: ds.time / np.timedelta64(1, "s"))
    )
    map_estimate = maximum_a_posteriori_estimate(posterior.sum("state"))

    labels, n_labels = scipy.ndimage.label(is_classified.sel(state=state))

    return [np.mean(map_estimate[labels == l]) for l in range(1, n_labels + 1)]


replay_linear_position_by_state = [
    get_replay_linear_position_by_state(
        ripple_number, is_classified, results, state="Hover"
    )
    for ripple_number in ripple_times.index[
        replay_info.Hover_replay_distance_from_actual_position > 30
    ]
]


replay_linear_position_by_state = np.asarray(
    list(itertools.chain.from_iterable(replay_linear_position_by_state))
)

In [141]:
replay_linear_position_by_state

array([ 93.78504514,   1.45990783, 314.28554804])

In [133]:
[
    blah
    for blah in ripple_times.index[replay_info.loc[:, "Hover_replay_speed"].values > 30]
]

[27]

In [127]:
ripple_times.shape

(193, 2)

In [142]:
replay_info.Hover_duration

ripple_number
1      0.046
2      0.000
3      0.000
4      0.000
5      0.000
       ...  
189    0.000
190    0.000
191    0.000
192    0.000
193    0.000
Name: Hover_duration, Length: 193, dtype: float64

In [121]:
epoch_key

('bon', 3, 6)

In [122]:
from src.analysis import get_replay_info

track_graph, _ = make_track_graph(epoch_key, ANIMALS)

replay_info = get_replay_info(
    results,
    ripple_spikes,
    ripple_times,
    position_info,
    track_graph,
    sampling_frequency,
    probability_threshold,
    epoch_key,
    classifier,
)

ripple_number
1      False
2      False
3      False
4      False
5      False
       ...  
189    False
190    False
191    False
192    False
193    False
Name: Hover_replay_speed, Length: 193, dtype: bool

In [106]:
from replay_trajectory_classification import ClusterlessClassifier

animal, day, epoch = epoch_key
data_type, dim = "clusterless", "1D"

model_name = os.path.join(
    PROCESSED_DATA_DIR, f"{animal}_{day:02}_{epoch:02}_{data_type}_{dim}_model.pkl"
)
classifier = ClusterlessClassifier.load_model(model_name)

In [84]:
import copy

import dask.bag as db
import networkx as nx
import numpy as np


def _distance_to_bin_centers(
    left_node,
    right_node,
    distance_left_node,
    distance_right_node,
    time_ind,
    copy_graph,
    place_bin_center_node_ids,
):
    node_name = f"actual_position_{time_ind}"
    copy_graph.add_node(node_name)
    copy_graph.add_edge(left_node, node_name, distance=distance_left_node)
    copy_graph.add_edge(node_name, right_node, distance=distance_right_node)
    distance_to_bin_centers = [
        nx.shortest_path_length(
            copy_graph, source=bin_center, target=node_name, weight="distance"
        )
        for bin_center in place_bin_center_node_ids
    ]
    copy_graph.remove_node(node_name)

    return distance_to_bin_centers


def _find_adjacent_nodes(nodes_df, linear_position):
    # Find the index of the nodes to insert between
    right_bin_ind = np.searchsorted(
        nodes_df.linear_position.values, linear_position, side="right"
    )
    left_bin_ind = right_bin_ind - 1

    # Fix indices that are exactly the maximum bin edge
    n_bins = nodes_df.linear_position.shape[0]
    right_bin_ind[right_bin_ind >= n_bins] -= 1

    # Fix for ones that fall into invalid track positions
    # Note: Need to test on different binnings
    # Another solution is to find the edge this falls on directly.
    not_same_edge = (
        nodes_df.edge_id.values[left_bin_ind] != nodes_df.edge_id.values[right_bin_ind]
    )
    right_bin_ind[not_same_edge] = right_bin_ind[not_same_edge] - 1
    left_bin_ind[not_same_edge] = left_bin_ind[not_same_edge] - 1

    # Get adjacent node names and distance
    left_node = nodes_df.reset_index().node_ids.values[left_bin_ind]
    right_node = nodes_df.reset_index().node_ids.values[right_bin_ind]

    distance_left_node = np.abs(
        nodes_df.loc[left_node].linear_position.values - linear_position
    )
    distance_right_node = np.abs(
        nodes_df.loc[right_node].linear_position.values - linear_position
    )

    return left_node, right_node, distance_left_node, distance_right_node


def get_distance_to_bin_centers(linear_position, decoder, npartitions=100):
    copy_graph = copy.deepcopy(decoder.track_graph_)
    linear_position = linear_position.squeeze()
    nodes_df = decoder._nodes_df.set_index("node_ids")
    place_bin_center_node_ids = (
        nodes_df.loc[~nodes_df.is_bin_edge].reset_index().node_ids.values
    )
    (
        left_node,
        right_node,
        distance_left_node,
        distance_right_node,
    ) = _find_adjacent_nodes(nodes_df, linear_position)

    left_node = db.from_sequence(left_node, npartitions=npartitions)
    right_node = db.from_sequence(right_node, npartitions=npartitions)
    distance_left_node = db.from_sequence(distance_left_node, npartitions=npartitions)
    distance_right_node = db.from_sequence(distance_right_node, npartitions=npartitions)
    time_ind = db.from_sequence(
        range(linear_position.shape[0]), npartitions=npartitions
    )

    return np.asarray(
        left_node.map(
            _distance_to_bin_centers,
            right_node,
            distance_left_node,
            distance_right_node,
            time_ind,
            copy_graph,
            place_bin_center_node_ids,
        ).compute()
    )


def gaussian_kernel(distance, bandwidth):
    return (
        np.exp(-0.5 * (distance / bandwidth) ** 2) / (bandwidth * np.sqrt(2.0 * np.pi))
    ) / bandwidth

In [88]:
distance_to_bin_centers = get_distance_to_bin_centers(
    replay_linear_position_by_state, classifier, npartitions=100
)

In [91]:
distance_to_bin_centers

array([[  0.        ,   2.91981567,   5.83963134, ..., 171.53259068,
        174.51775074, 177.50291081],
       [  0.        ,   2.91981567,   5.83963134, ..., 171.53259068,
        174.51775074, 177.50291081],
       [  0.        ,   2.91981567,   5.83963134, ..., 171.53259068,
        174.51775074, 177.50291081],
       ...,
       [  8.759447  ,   5.83963134,   2.91981567, ..., 162.77314367,
        165.75830374, 168.74346381],
       [  0.        ,   2.91981567,   5.83963134, ..., 171.53259068,
        174.51775074, 177.50291081],
       [ 72.99751757,  70.0777019 ,  67.15788623, ..., 101.4591405 ,
        104.44430057, 107.42946064]])

In [92]:
distance_to_bin_centers.shape

(31, 97)

In [98]:
classifier.place_bin_centers_[classifier.is_track_interior_].squeeze()

array([  1.45990783,   4.3797235 ,   7.29953917,  10.21935484,
        13.13917051,  16.05898618,  18.97880184,  21.89861751,
        24.81843318,  27.73824885,  30.65806452,  33.57788018,
        36.49769585,  39.41751152,  42.33732719,  45.25714286,
        48.17695853,  51.09677419,  54.01658986,  56.93640553,
        59.8562212 ,  62.77603687,  65.69585253,  68.6156682 ,
        71.53548387,  89.4574254 ,  92.38149279,  95.30556019,
        98.22962758, 101.15369497, 104.07776236, 107.00182976,
       109.92589715, 112.84996454, 115.77403193, 118.69809932,
       121.62216672, 124.54740709, 127.47382046, 130.40023382,
       133.32664718, 136.25306054, 139.1794739 , 142.10588726,
       145.03230063, 147.95871399, 150.88512735, 153.81154071,
       156.73795407, 159.66436744, 162.5907808 , 165.51719416,
       168.44360752, 171.37002088, 174.29643425, 177.22284761,
       180.14926097, 183.07567433, 186.00208769, 188.92850105,
       191.85491442, 209.81046132, 212.79514177, 215.77